# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='/Users/ben/webcrawler/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

<ipython-input-5-de39ffb3f06f>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='/Users/ben/webcrawler/chromedriver')


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 聯亞藥12／14召開股東臨時會　討論申請上市、現金增資案

▲聯亞藥今日公告將於12月14日召開股東臨時會。（圖／翻攝自聯亞藥官網）
記者吳康瑋／綜合報導 
聯亞藥（6562）今（25）日發重訊指出，公司將於12月14日召開股東臨時會，除將報告公司各方面誠信、道德準則外，預計將與現場股東針對申請股票上市案、及現金增資暨原股東全數放棄優先認股權利案，進行詳細討論。
聯亞藥表示，欲參加股東會的投資人，依公司法第192條之1規定，持有已發行股份總數百分之一以上股份的股東，向公司提出董事候選人名單，提名人數不得超過董事應選名額9席(含獨立董事4席)。凡有意提名之股東務請於11月15日下午5點前，寄回公司本部以利董事會審查及回覆結果。
據了解，聯亞藥於民國103年7月31日由母公司聯亞生技分割設立，是由聯亞生技開發股份有限公司（簡稱聯亞生技）之非抗體蛋白質藥品開發、小分子藥品開發及醫藥製造等綜合醫藥業務分割新設之子公司，且著重在生物相似藥品及小分子藥品的開發。
聯亞藥今年6月23日，以每股30元登錄興櫃，僅僅掛牌第一天，股價就快速飆上百元俱樂部，爆出上萬張成交量，第二天漲勢仍不停歇，盤中漲幅擴大超過5成，盤中還曾衝上300元。不過隨著母公司聯亞生技8月16日接獲旗下研發的新冠疫苗，未通過衛福部緊急授權的消息，當天仍在興櫃市場交易的聯亞藥股價跌破200元，短短2分鐘跌幅超過16％，最低跌至140元，高低點價差近70元。
不過，聯亞生技沒有因此放棄，後續在9月中，向衛福部食藥署就緊急使用授權（EUA）審查結果重新提出訴願，並且繼續推進二期臨床試驗，團隊預計完成數據分析，再另外召開解盲記者會公佈完整數字，且不排除在完成青少年部分後，再次申請EUA。
針對聯亞藥股價，雖然已沒有當初登興櫃時，飆漲到200、300元的驚人幅度，可是聯亞藥在這短短的四個月內，也成長超過五成，若當初登興櫃前，提早買進聯亞藥股票，以今日收盤價81.9元來看，每股帳面可賺進新台幣5.19萬元。
----------------------------------------------------------------------

[1] 9月證券劃撥餘額創

記者吳珍儀／綜合報導
台股今 (25) 日盤中一度挫逾百點，跌破5日線，但在電子權值股、貨櫃三雄、鋼鐵回神下，加權指數午盤前翻紅，5日線失而復得，終場收在16894點，上漲5點或0.03%，成交量2666.34億元。《雲端最有錢》專家表示，台股是否上攻，必須觀察「2個關鍵點」。
財經專家阮慕驊表示，最近台股最大的問題，量一直放不出來。但是其實櫃買已經在突破了，櫃買上周的均量都在9000億，對櫃買來講就是一個不錯的量。
阮慕驊認為，市場的資金很聰明，知道權值股跟大型的上市股票拉不太動，尤其是有外資的賣壓，所以開始把資金轉去中小型，比較有作夢行情的電子股，去拉櫃買。但大盤也沒有太差，上週加權指數還有小漲，櫃買上週漲得更兇，漲了3%。
阮慕驊指出，其實資金是在轉戰一些中小型股票去，所以在這樣的情況下，大盤到底能不能突破往上走，就要看量。如果量可以突破3000億持續放大出來，那問題就不大。
從大盤來看，從7月以來出現一個大箱型結構，高點在18000，低點在16000，上下2000點大概10%左右的區間，那目前約在中間17000的位置，所以這兩天大盤就沒有像一開始從16162點反彈這麼強勁，就是因為已經處在中間的位置。阮慕驊再次提到，大盤能不能往上突破，投資者必須觀察量，盤中的預估量是不是能到3000億，或是能不能往上走到3500億甚至更多，就是一個很重要的關鍵。
阮慕驊表示，不用天天都每種類股齊漲，但一定要有類股輪動的味道，航運、鋼鐵、塑化，這三個主要的；主攻部隊則是IC設計、半導體、晶圓代工這些族群，能不能持續發揮推力，就是很關鍵的點。
----------------------------------------------------------------------

[12] 電信業者推真人導遊視訊　線上就可漫步巴黎街頭

▲亞太電信結合MyProGuide推出線上真人直播導覽旅遊服務。（圖／亞太電信提供）
記者陳俐穎／台北報導
疫情衝擊，許多人已經好久沒有出國旅遊，電信業者就看到了這項需求，亞太電信今宣布與MyProGuide推出線上真人直播導覽旅遊服務，藉由5G大頻寬、低延遲特性，實現零時差、雙向互動的旅遊景點直播。用戶僅需開啟手機、平板或電腦進行視訊連線，就能一秒立刻飛出國，由當地專業導遊帶你暢遊東京表參道、巴黎塞納河、倫敦大笨鐘等世界知名景點，一解久久

● 鍾文榮／科普經濟學作家、資深產業分析師
這個月初主計長說蚵仔麵線一碗20元的說法依舊餘波蕩漾，行政院長當時為主計長緩頰，說他小事糊塗、大事清楚。有了比較基礎之後，行政院長事後回應立委質詢時馬上說蚵仔麵線一碗約60元，算是扳回一點行政院對於物價敏感顏面，至少沒有「乃覺三十里」。
蚵仔麵線與物價
事實上，主計總處的「物價統計資料庫」中針對「外食類」的「其他家外食物」還真的有「蚵仔麵線」這一項，這算是一種巧合嗎？只是，「蚵仔麵線」是反映物價的「果」，而不是影響物價的「因」，但行政院長怎能推說主計長對物價這事只是「小事糊塗」呢？
▲在主計總處的「物價統計資料庫」中有「蚵仔麵線」這一項。（圖／主計總處「物價統計資料庫」）
國際紙漿已經連漲一陣子了，前幾天把家裡不用的廢紙打包賣給回收廠，十月初的報價竟然高達4.2元。
根據主計總處資料庫統計，食物類CPI年增率在六月時已經超越CPI年增率，這也是民眾對物價最有感之處。上週日丈母娘來我家吃飯，她翻攪了一下我家的電冰箱，竟然抱怨我家電冰箱找不到青蔥可以炒菜。我只好回說菜價貴，連香菜也貴到我買不下手。
接下來她就一直嘟嚷著說，她到菜市場僅能用10元買到「兩根蔥」，我真的沒聽錯，10元買「兩根蔥」，她繼續抱怨她的買菜經，菜也貴肉也貴，乾脆吃白飯好了。她可能沒料到，我家的蛋炒飯已經很久沒加青蔥提味了！
婆婆媽媽們到菜市場買菜，當然是有感，買菜當然不會「小事糊塗」。前幾天聽聞飼料大漲，想必沒多久肉價也會漲價。當菜價、肉價上漲時，我料定我家餐桌上的晚餐，應該會全面換成根莖類蔬菜，絕對不會「何不食肉糜」！
官方對於物價的說詞只是「安慰人心」
但我們聽到的官方消息是物價還在「合理範圍」內，是「短期」現象，目前沒有通貨膨脹問題。但問題是，即使主計總處、經濟部、中央銀行紛紛表示目前沒有通貨膨脹問題，但為何行政院卻啟動穩定物價小組，要嚴查哄抬物價的問題？
大家請注意一下行政院的用詞。一方面出現「穩定」物價小組，一方面要嚴查「哄抬」物價問題，請問，這個在語意上已經編碼過的訊息，傳達出何種趨勢呢？
其實，就是行政院暗示了物價確實出現了上漲問題，然民眾已經形成了預期心理，只要這個預期心理一旦大眾化之後，物價問題就嚴重了，而且真的可能形成通貨膨脹，這當然是政府不容預見之事！
基於預期心理的影響，我才會說官方對於物價的說詞，基本上只是「安慰人心」，卻無意間釋

▲大立光執行長林恩平今（25）日發重訊指出，董事會決議通過買回公司庫藏股1342張。（資料照／記者高兆麟攝）
記者吳康瑋／綜合報導
手機鏡頭大廠大立光（3008）終於決定實施庫藏股了！大立光今（25）日發重訊宣布，董事會為維護公司信用及股東權益，決議通過買回公司庫藏股1342張，買進的價格區間為每股2,025元至3,300元，且當公司股價低於所定區間價格下限時，仍將繼續執行買回股份。大立光預計此次庫藏股將實際動用約44億元，這也是上市公司近期大手筆護盤動作。
大立光強調，公司將從明天開始到12月24日間，買回庫藏1342張普通股，占公司已發行股份1％，且買回股份所需金額上限僅占公司流動資產3.57%，並該決議已經通過出席董事6人同意，大立光表示，董事會已考慮公司財務狀況，且買回上述股份，並不影響公司資本維持。
以大立光今天收盤價2015元計算，雖然低於庫藏股預計實施的買進價下限，但依法還是可以買進，以此價格計算，如果買進1342張，預計要花27億元左右。如果買到上限價3300元，大概要花44.2億元。
身為國內鏡頭廠龍頭，大立光股價卻從年初的3360元，一路跌到近期的2千元上下，投資人頗有怨言，也關切大立光是否會實施庫藏股。在本月初的法說會上，執行長林恩平坦言，這部分有在考慮。
結果，大立光董事會今天真的做成決議，要實施庫藏股，回應股民期望。
法說會當天，林恩平也預告，公司的「專利武器」也是時候該用出來了，該用時，就一定會用。
林恩平指出，大立光其實4年前就在做6P鏡頭，不過到現在還在交貨，而且還有一定的出貨量，不過客戶放規格給競爭對手，但對手的規格一直跟不上，直到對手在今年已經跟在規格上追上，因此大立光不排除會發動專利戰，來避免市佔率下滑，且公司聚焦高階鏡頭技術，也有相當多專利，因此專利比起保護，更可以當作積極的武器來用。
----------------------------------------------------------------------

[27] 富邦台灣半導體ETF追蹤指數調整　新增昇佳電子、刪除松翰

▲ICE FactSet台灣核心半導體指數前10大成分股今年漲幅。（資料來源／CMoney）
記者陳心怡／台北報導
-----------------------------------------------------------

▲國泰世華銀ATM出包事件，金管會不允許再發生。（圖／取自免費圖庫pixabay）
記者陳依旻／台北報導
國泰金控旗下國泰世華銀行日前連續當機兩天，金管會主委黃天牧今（25）日表示，已請國泰世華銀一周內說明，未來類似情況不允許再發生。
立委今日在財委會上關切國泰世華網銀為了更新升級系統，導致ATM連續當機兩天，影響4萬多筆交易，部分民眾也投訴銀行客服態度不佳，要求金管會監督銀行數位系統升級過程中，不能影響顧客的權益。
對此，黃天牧回應，根據金管會了解，這次出問題的原因不是核心系統，而是ATM系統，銀行局已發函給國泰世華銀，請其一周提出當機說明，未來不允許再發生，更一度直言，國泰世華銀此次，非核心系統升級也會出現這種狀況，讓他「不能夠接受」。
----------------------------------------------------------------------

[37] 台股5日線失而復得！航海王回神　指數漲5點收16894

▲台股終場收在16894點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
台股今 (25) 日開低走低，盤中一度挫逾百點，跌破5日線，但在電子權值股、貨櫃三雄、鋼鐵回神下，加權指數午盤前翻紅，5日線失而復得，終場收在16894點，上漲5點或0.03%，成交量2666.34億元。
蘋概三王中，台積電（2330）以593元作收、下跌1.17％；鴻海（2317）以108元作收、上漲0.47%；大立光（3008）以2015元作收、下跌0.49％。
其他大型權值股除了台積電、大立光、南亞、台化、中華電下跌，其他都上漲。
今日上市類股漲跌互見，表現較好為運輸類股，上漲4.06%；表現較差為百貨類股，下跌2.77%。
傳產股方面，貨櫃三雄強勢回神，長榮大漲逾6%，陽明漲逾5%，萬海漲逾3%，其他運輸股，包括中櫃、志信也都有4%以上漲幅。
元宇宙概念股宏達電（2498）盤中攻上54.5元、漲幅近4%，但買盤追價力道不足，股價隨之翻黑，最低來到51.1元、跌幅逾2%，終場以53元作收、上漲0.95%。
美國電動車大廠特斯拉傳出將更換電池供應商，台廠美琪瑪（4721）、康普（4739）、聚和（6509）有望受惠，今天觸及漲停142元、140.5元、67.8元。
---------------------------------------

▲成霖集團創辦人歐陽明先生（圖右）23日晚間辭世。（資料照／中央大學提供）
記者林淑慧／台北報導
水龍頭大廠成霖企業今（25）日公告，創辦人暨榮譽董事長歐陽明23日晚間因病辭世，公司將更換法人董事代表；歐陽明由建材貿易公司起家，曾於2009年獲得台灣《安永企業家獎》2009 年度大獎，對其辭世各界備感不捨。
歐陽明為白手起家，於1979年成立貿易公司經營建材，而後毅然投入水龍頭研發與產製，曾任成霖企業股份有限公司董事長、總經理、榮譽董事長，在其領導下，成霖一躍成為亞洲規模最大的水龍頭工廠，並在數個領域中以技術領先全球。
歐陽明對成霖的營運主張採取代工與自有品牌共榮策略，且經營範疇由水龍頭擴展到衛浴，從北美橫跨到大陸、歐洲。更讓成霖成為第一家同時在海峽兩岸上市的衛浴公司，更帶動亞洲衛浴產業的蓬勃發展。
成霖於2019年完成二代接班，由歐陽明長子歐陽玄，接任董事長。
----------------------------------------------------------------------

[46] 摩鐵變住宅！建商搶布局R15站　未來5年房價估坐3望4

記者陳建宇／高雄報導
高雄捷運R15生態園區站區段位置優越，而生態園區站還有許多如摩鐵、停車場等可供開發的土地，也變成許多建商鎖定的購地目標，2年前京城建設購入的「玫瑰精品汽車旅館」近期就已歇業，據了解將進行開發，未來房價將站上每坪40萬元，等於未來5年內房價將漲10萬元。
▲高捷生態園區站周邊摩鐵將開發為住宅，房價將站上4字頭。（圖／記者陳建宇攝）
北高雄是目前高雄的發展重心，許多建商都在北高雄找地蓋房子，而生態園區站因為因為介於巨蛋及高鐵中間，且周邊可用土地仍多，因此成為建商鎖定的目標，在地龍頭建商京城建設就在2019年砸下16.19億元，購入「玫瑰精品汽車旅館」，該地面積1057.57坪，換算單價每坪153.1萬元。
▲生態園區站附近還有不少土地可用，成為建商鎖定的目標。（圖／記者陳建宇攝）
▲該站周邊的商4土地土地有每坪200萬元的行情，住4土地也要每坪100萬元。（圖／記者陳建宇攝）
張鎧麟指出，由於日前鑫龍騰以每坪200萬元的價格買下高鐵站附近土地，因此目前生態園區站附近的商4土地開價已經超過200萬元，成交價會落在每坪200萬元上下，而住4的容積率沒那麼高，前一陣子有建商以每坪92萬元

▲地震頻傳，新竹科學園區正常運作。（圖／ETtoday資料照）
記者林淑慧／台北報導
國內近期地震頻傳，經濟部長王美花今（25）日於立法院表示，昨天發生的規模6.5地震，經濟部第一時間回報水電供應均穩定，目前科學園區等高科技業沒有災損情況。
另外，針對年底的四大公投議題，王美花回應指出，近期會與各界包括環團多加說明，再將發電期程、藻礁生態保護等政策對外說明，強調對外說明政策本來就是經濟部該做的，會與各界多溝通，不會只邀請環團與會。
國內昨日下午1時11分、12分左右，發生芮氏規模6.5地震，地震深度66.8公里，震央位於宜蘭縣政府南方22.7公里（位於宜蘭縣南澳鄉)，造成台北捷運全線一度停駛，經濟部也緊盯台電核能發電廠，以及各科學園區內的電力供應問題。
經長王美花表示，地震有觸發核一、二廠地震儀動作，經台電第一時間回報，確認機組都正常運作。各水力、火力機組、壩堰及輸電設備正常運轉中，輸、配電系統電網運作正常。
各科學園區內的高科技業，屬科技部權責，經跨部會資訊交換，昨日停電對高科技業沒有衝擊，產線及產能均正常運作。
----------------------------------------------------------------------

[57] 台灣坎城創意節最大贏家！好評推出信任幸福二部曲引發共鳴　網：好揪心

房產中心／綜合報導
婚姻真的像童話故事裡的王子與公主，天天過著幸福快樂的日子嗎？信義房屋繼先前「信任幸福篇」感動百萬觀眾，狂掀「婚」、「不婚」熱議，甚至前進世界級電影殿堂，榮獲2021年坎城創意娛樂獅的全場最大獎，創下台灣品牌行銷業界最高獲獎新紀錄。近期延續好評推出「信任幸福二部曲」，再度由視帝姚淳耀、實力派演員陳美月刻畫婚後最真實的生活樣貌，再掀年輕世代的大勢議題「生」、「不生」超揪心劇情火速引發網友共鳴。
這次信義房屋「信任幸福二部曲」以男性視角出發，由姚淳耀飾演開設照相館拍攝全家福照片為職業的攝影師，記錄不同家庭成員的幸福瞬間，然而每天來來去去的家庭都有各自的難題，包括攝影師與自己的太太也正面臨生育難題。
原來結婚後的攝影師與太太為了順利懷孕，努力嘗試各式各樣的方法都不見成效，從嚴格控制飲食到原本應該是最快樂的親密關係都變成必須按表操課，直到一次攝影師在收到訊息後，沒有準時回家「做人」，引爆太太長期壓力與不滿，兩人大吵一架。
隔

▲其他地方台商收到限電通知書（非昆山） 。（圖／台商提供）
記者任以芳／昆山報導
大陸「限電」政策持續進行，「能耗雙控」的政策也讓各省台資企業工廠受衝擊。《東森新媒體ETtoday》記者追蹤限電後續，走訪有十萬台商的江蘇昆山，當地老台商受訪表示，昆山有很多大型台資企業大多自備發電機或太陽能板設備，至少能應付20%用電量，但「限電」對於整體產業衝擊不小，大家都很擔心，可能到12月年底都不會停止「限電」政策。
昆山有十萬台商，長年投資密集扎實。根據當地政府資料顯示，截至2019年底，累計批准台資項目5196個，投資總額超620億美元，註冊資本319億美元。台資經濟貢獻昆山地區生產總值的30%，工業總產值的50%。
▲昆山老台商陳順浪 。（圖／記者任以芳攝）
正因為許多台商在昆山扎根太深，對江蘇GDP貢獻良多，不少「規模以上」的大型台資企業沒有被「拉閘限電」，但一樣得配合政策，縮減用電功率。昆山老台商陳順浪《東森新媒體ETtoday》訪問表示，「我們用電量自主降到6到7成，其他用發電機補足，調整員工用電離峰期間上班，影響不大。」
談到其他中小型企業概況？陳順浪說，「外面報導有些過頭，謠傳昆山強制拉閘限電，這是很嚴重事情，工廠在製造過程中產品會損壞，容易引起安全問題，拉閘是非常不得已情況下才會實行，當地政府一般不會做這個動作。」‌‌
另外，他進一步介紹，比如白天室外溫度升高，工廠必須調整用電。尤其是空調佔製造業40%用電，為了應對限電，通常工廠會把空調關掉；其次調整上班人力，避開白天高峰用電，挪移晚上開工。
▲各地台商節約用電 。（圖／記者任以芳攝）
談到「限電」措施有無解辦法？昆山第一代老台商吳澧淦受訪時也直言，「限電對於大陸整體產業真的影響很大，如果之後適度調整電價，或許就能解決部分現況。」
不願具名老台商指出，昆山「限電令」暫時到十月底，雖然當地不准私自用發電機，但大陸或台資規模以上大型企業例外，否則對上下游產業鍊影響十分巨大。不少台商憂心直到12月底依然常態限電。‌大家心裡也清楚，這波限電不是針對台商，而是配合整體減碳政策，只能配合到底。
----------------------------------------------------------------------

[70] 「誰家有剩餘電，我買！」　無錫台商互支援缺電潮

▲配合限電，無錫不少工廠白

記者陳依旻、高兆麟／台北報導
地牛翻身「震」驚全台，今（24）日下午1時11分左右東北地區發生中型有感地震，規模6.5，不但民眾嚇了一跳，各大科技廠也都緊急應變，像是聯電，有些機台立刻啟動保護裝置停機，聯電表示，正陸續復機中，營運沒重大影響；世界先進也在第一時間按標準作業程序(SOP)進行各廠人員疏散，確認安全無虞後，目前人員皆已返回工作崗位。初步評估，公司的生產及營運未受影響。
▲聯電受地震影響，部分機台停機。（圖／聯電提供）
6.5地震震驚全台，花蓮縣地區最大震度3級、宜蘭縣地區最大震度3級、南投縣地區最大震度2級、台中市地區最大震度2級、新竹縣地區最大震度1級、彰化縣地區最大震度1級、桃園市地區最大震度1級、雲林縣地區最大震度1級。
----------------------------------------------------------------------

[81] 移工匯兌公司列入金保法　評議中心將提供這服務！

▲移工匯兌公司列入金保法範圍，未來移工匯款如遇爭議，可至評議中心申訴。圖為評議中心董事長林志潔。（圖／記者陳依旻攝）
記者陳依旻／台北報導
外籍移工匯兌公司將被列入金融消費者保護法規定的金融服務業，日後移工若匯款時遇到爭議，可向金融評議中心申請調處，評議中心董事長林志潔表示，未來會提供不同語言的服務，要讓移工有個友善環境可以申訴。
行政院洗錢防制辦公室以及財團法人中華民國證券暨期貨市場發展基金會日前主辦「公開發行公司金融犯罪之防制」研討會，會中邀請陽明交通大學特聘教授林志潔擔任主講人。
林志潔同時也是金融消費評議中心董事長，林志潔指出，過去移工多走不合法的地下匯兌，目前統振已經沙盒通過落地，移工多了一個管道可以匯款回母國。
但有外界擔憂，移工如果匯款出去，母國沒有拿到錢，或是在使用統振的過程中資訊外洩，移工可不可以請求評議，以及統振是否要納入金保法？她的主張是要納入，因為雖然統振不是金融業但已適用電子支付條例，目前金管會也將統振列入金保法名單之中。
林志潔進一步表示，好在統振有保護弱勢的決心，評議中心也願意承擔，未來會提供不同語言，讓移工來申訴的時候有友善環境。
過去多數外籍移工無法在銀行營業時間前往匯款，且銀行匯款手續費相對較高，統振及香港商易安聯2019年以行動裝置及超商繳款等便捷、低廉創新模式，向金管會申請進沙盒實驗獲准，於

▲macOS Monterey下周上線。（圖／取自9to5mac）
記者陳俐穎／綜合報導
蘋果日前在秋季發表會公布了全新Macbook Pro系列，一如以往，也將搭載最新作業系統，目前已知macOS最新版 ：Monterey，將在本週二（26日）推出，同一時間也將發布 iOS 15.1。
新的 macOS Monterey 具備全新 FaceTime，支援空間音訊、專注模式、人像模式。其中，所謂的空間音訊，就是每個人的聲音就像來自他們在螢幕上的位置；人像模式則是讓視訊畫面聚焦於人，讓背景模糊，類似拍照景深效果。
而在疫情年代，也可以透過SharePlay 與朋友一起觀看影片、聽音樂，就像是在同一個空間。
快速備忘錄功能，讓你不管在哪，或在 Mac 上做什麼，都能快速記下靈感構思，感覺就像直接做筆記在你正用著的 app 裡。
捷徑 App，讓你最常做的事，可以簡單做到自動化。將需要多個步驟才能完成的事簡化成一個步驟，或使用「捷徑」編輯器，將常用的App 和服務連結在一起，要執行捷徑時，從 Dock、選單列、Finder、Spotlight，甚或透過 Siri 都可以。
自 6 月以來，蘋果一直在對 macOS Monterey 進行 Beta 測試，根據蘋果官網訊息，確認 macOS Monterey 將於台灣時間10月26日推出，而iOS 15.1 也可能會同時發布。
目前支援 macOS Monterey 的裝置型號為2015 年及以後的 MacBook、2015 年初及之後的 MacBook Air、2015 年初及之後的 MacBook Pro、2014 年末及更新的 Mac mini、2015 年末及更新的 iMac、2017 年及更新的 iMac Pro、2013 年末及更新的 Mac Pro。
----------------------------------------------------------------------

[91] 南韓將申請「加入CPTPP」...時間點曝光！憂遭日本逼開放福島核食

▲南韓最快將於11月初決定是否申請加入CPTPP，圖為釜山港。（圖／達志影像）
記者羅翊宬／編譯
繼中國與台灣先後於9月申請加入「跨太平洋夥伴全面進步協定」（以下稱CPTPP），為了不在談判條件上處於劣勢，在經過一番計算後，南韓政府最終決定最